In [ ]:
!pip install tensorflow==2.10.0

In [ ]:
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import activations
#from tensorflow.keras import optimizers

In [ ]:
# Model configuration
img_width, img_height = 224, 224
batch_size = 50
no_epochs = 15
no_classes = 2
validation_split = 0.1131
verbosity = 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
PATH = '/content/drive/MyDrive/Brain22/BDT/csvfile/'
train_false=pd.read_csv(PATH + 'train_false.csv',header=None)
train_true=pd.read_csv(PATH + 'train_true.csv',header=None)

In [ ]:
test_false=pd.read_csv(PATH + 'test_false.csv',header=None)
test_true=pd.read_csv(PATH + 'test_true.csv',header=None)
val_false=pd.read_csv(PATH + 'val_false.csv',header=None)
val_true=pd.read_csv(PATH + 'val_true.csv',header=None)

In [ ]:
print(train_true.shape)
print(train_false.shape)

In [ ]:
import numpy as np
true = train_true
false = train_false
x=np.array(true)
y=np.array(false)
train=np.vstack((x, y))
print(train.shape)

In [ ]:
train_label=np.zeros(202)
train_label=train_label.astype(int)
train_label[0:124]=1 #Tumor = 1, No-Tumor = 0
print(train_label)


In [ ]:
encoded_array = np.zeros((train_label.size, train_label.max()+1), dtype=int)
encoded_array[np.arange(train_label.size),train_label] = 1 
train_label=encoded_array
print(train_label)

In [ ]:
true = test_true
false = test_false
x=np.array(true)
y=np.array(false)
test=np.vstack((x, y))
print(test.shape)

In [ ]:
test_label=np.zeros(26)
test_label=test_label.astype(int)
test_label[0:16]=1 #Tumor = 1, No-Tumor = 0
print(test_label)

In [ ]:
encoded_array = np.zeros((test_label.size, test_label.max()+1), dtype=int)
encoded_array[np.arange(test_label.size),test_label] = 1 
test_label=encoded_array
print(test_label)

In [ ]:
true = val_true
false = val_false
x=np.array(true)
y=np.array(false)
val=np.vstack((x, y))
print(val.shape)

In [ ]:
val_label=np.zeros(25)
val_label=val_label.astype(int)
val_label[0:15]=1 #Tumor = 1, No-Tumor = 0
print(val_label)

In [ ]:
encoded_array = np.zeros((val_label.size, val_label.max()+1), dtype=int)
encoded_array[np.arange(val_label.size),val_label] = 1 
val_label=encoded_array
print(val_label)

In [ ]:
train_label.shape

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 1)
axes.imshow(test[0].reshape(224,224), cmap='gray') 
axes.set_title('Input')
  
fig.suptitle(f'Ekush target = {test_label[0]}')
plt.show()

In [ ]:
print(train_label.shape)

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

# weights = {0:10, 1:1}
weights = {0:1.5897, 1:1}

In [ ]:
if K.image_data_format() == 'channels_first':
    train = train.reshape(train.shape[0], 1, img_width, img_height)
    test = test.reshape(test.shape[0], 1, img_width, img_height)
    val = val.reshape(val.shape[0], 1, img_width, img_height)
    input_shape = (1, img_width, img_height)
else:
    train = train.reshape(train.shape[0], img_width, img_height, 1)
    test = test.reshape(test.shape[0], img_width, img_height, 1)
    val = val.reshape(val.shape[0], img_width, img_height, 1)
    input_shape = (img_width, img_height, 1)


# Parse numbers as floats
#print(train)
train = train.astype('float32')
test = test.astype('float32')
val = val.astype('float32')
#print(train)

# Normalize data
train = train / 255
test = test / 255
val = val / 255

# Convert target vectors to categorical targets
# train_label = to_categorical(train_label, no_classes)
# test_label = to_categorical(test_label, no_classes)

# Create the model
model = Sequential()
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='softmax', name='hello'))

print(model.summary())

# Compile the model
model.compile(loss=tensorflow.keras.losses.binary_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(2e-5),
              metrics=['acc'])

# Fit data to model
model.fit(train, train_label,
          batch_size=batch_size,
          epochs=10,
          verbose=verbosity, 
          class_weight=weights,validation_data = (val, val_label))


In [ ]:
model.fit(train, train_label,
          batch_size=batch_size,
          epochs=5,
          verbose=verbosity, 
          class_weight=weights,validation_data = (val, val_label))

In [ ]:
model.save("/content/drive/MyDrive/Brain2/BDTCS/csvfile/BDT35.h5")

In [ ]:
test_prob = model.predict(test)

In [ ]:
pd.DataFrame(test_prob).to_csv("/content/drive/MyDrive/Brain2/BDTCS/csvfile/testprobBDT35.csv")

In [ ]:
from sklearn.metrics import confusion_matrix
y_prediction = np.argmax(test_prob,axis=1)
print(y_prediction)

y_test=test_label
y_test = np.argmax(y_test,axis=1)
# #Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_prediction)
print(result)

In [ ]:
print(y_prediction)

In [ ]:
pd.DataFrame(y_prediction).to_csv("/content/drive/MyDrive/Brain2/BDTCS/csvfile/predclassBDT35.csv")

In [ ]:
pd.DataFrame(result).to_csv("/content/drive/MyDrive/Brain2/BDTCS/csvfile/cmBDT35.csv")

In [ ]:
from sklearn.metrics import classification_report

# #Create confusion matrix and normalizes it over predicted (columns)
result = classification_report(y_test, y_prediction,output_dict=True)
print(result)

In [ ]:
pd.DataFrame(result).to_csv("/content/drive/MyDrive/Brain2/BDTCS/csvfile/crBDT35.csv")